## A simple IFR example

Load packages.

In [1]:
import os
import sys
import numpy as np
import torch

sys.path.insert(0, '../')
from ifr import IFR
import utils

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
ifr_model = IFR(scale=2, maxiter=20,zero_mean=True,
                trunc=True,rand_pa=False, kp_nb=False,encoder_id=4)

In [3]:
def do_transform(p0, x):
    # p0: [N, 3]
    # x: [1, 6], twist-params
    g = utils.exp(x).to(p0) # [1, 4, 4]
    p1 = utils.transform(g, p0)
    igt = g.squeeze(0) # igt: p0 -> p1
    return p1, igt


# load data
p0 = np.load('./p0.npy')
p1 = np.load('./p1.npy')

# randomly set the twist parameters for the ground truth pose
x = np.array([[0.57, -0.29, 0.73, -0.37, 0.48, -0.54]])

p1_pre, igt = do_transform(torch.from_numpy(p1[np.newaxis,...]), torch.from_numpy(x)[np.newaxis,...])
p1_pre = p1_pre.numpy()[0,:,:]
print('GT Transform', igt)

GT Transform tensor([[[ 0.7150, -0.6970, -0.0543, -0.4716],
         [ 0.5443,  0.6037, -0.5824,  0.4599],
         [ 0.4388,  0.3868,  0.8111, -0.4687],
         [ 0.0000,  0.0000,  0.0000,  1.0000]]], dtype=torch.float64)


In [4]:
print('Input Shape without downsample', p0.shape, p1_pre.shape)
estimated_pose = ifr_model.register(p0, p1_pre)[0,:,:]
print('\n\n Solved:\n',estimated_pose)

print('Error Matrix:\n', estimated_pose@igt.numpy()[0,:,:])

Input Shape without downsample (103974, 3) (76241, 3)


 Solved:
 [[ 0.7021345   0.5471499   0.45566896  0.3461575 ]
 [-0.70915663  0.5949184   0.37837538 -0.44514206]
 [-0.06405792 -0.58881193  0.8057283   0.6202873 ]
 [ 0.          0.          0.          1.        ]]
Error Matrix:
 [[ 9.99771315e-01  1.71827249e-02  1.27326961e-02  5.30889790e-02]
 [-1.71702523e-02  9.99851461e-01 -1.07892063e-03 -1.44913672e-02]
 [-1.27496393e-02  8.59870553e-04  9.99918866e-01  2.08872309e-03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [5]:
# vis (visualization is on downsampled for fast draw)
p0_hat = (estimated_pose[:3,:3]@p1_pre.T+estimated_pose[:3,(3,)]).T

import ipyvolume as ipv

def plot(x1,x2):
    fig = ipv.figure()
    scatter = ipv.scatter(x1[:,0], x1[:,1], x1[:,2],np.array([255,0,0]))
    scatter = ipv.scatter(x2[:,0], x2[:,1], x2[:,2],np.array([0,0,255]))
    ipv.show()

In [8]:
plot(p0, p1_pre)
plot(p0, p0_hat)